# Imports

In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import math
import requests
import time

# Helper Functions

ssdfgsd

# Notebook Config

# Selenium

## Selenium W/O Functions

In [14]:
driver = webdriver.Chrome()
# acessar o site Vivareal
driver.get('https://www.vivareal.com.br/')

In [15]:
# Selecionar Alugar
alugar = 'rent'
compra = 'sale'

select_element01 = driver.find_element(By.CLASS_NAME, 'js-select-business')
select_object01 = Select(select_element01)
select_object01.select_by_value(alugar) # alguar

In [16]:
# Selecionar Casa
casa = 'HOME|UnitSubType_NONE,SINGLE_STOREY_HOUSE,VILLAGE_HOUSE,KITNET|RESIDENTIAL|HOME'
apartamento = 'APARTMENT|UnitSubType_NONE,DUPLEX,LOFT,STUDIO,TRIPLEX|RESIDENTIAL|APARTMENT'

select_element02 = driver.find_element(By.CLASS_NAME, 'js-select-type')
select_object02 = Select(select_element02)
select_object02.select_by_value(apartamento) # apartamento

In [17]:
# Bairro a ser procurado
bairro = 'moema'
full_address = bairro + ', São Paulo - SP'

select_element03 = driver.find_element(By.CLASS_NAME, 'js-input')
select_element03.send_keys(full_address)

driver.implicitly_wait(5)

search_bairro = driver.find_element(By.CLASS_NAME, 'js-location-name')
search_bairro.click() 

## Selenium Functions

### Functrions

In [19]:
def main_site():
    options = webdriver.ChromeOptions() 
    options.add_argument("start-maximized")
    driver = uc.Chrome(options=options)

    # driver = webdriver.Chrome(chrome_options=chrome_options)

    # driver = webdriver.Chrome()
    # acessar o site Vivareal
    driver.get('https://www.vivareal.com.br/')

    time.sleep(2)

    return driver


def select_purpose(driver, purpose):
    # Selecionar Alugar
    # alugar = 'rent'
    # compra = 'sale'

    select_element01 = driver.find_element(By.CLASS_NAME, 'js-select-business')
    select_object01 = Select(select_element01)
    select_object01.select_by_value(purpose)

    time.sleep(2)

def select_type(driver, type):
# Selecionar Casa
    if type == "Casa":
        type_ = 'HOME|UnitSubType_NONE,SINGLE_STOREY_HOUSE,VILLAGE_HOUSE,KITNET|RESIDENTIAL|HOME'
    elif type == "Apartamento":
        type_ = 'APARTMENT|UnitSubType_NONE,DUPLEX,LOFT,STUDIO,TRIPLEX|RESIDENTIAL|APARTMENT'

    select_element02 = driver.find_element(By.CLASS_NAME, 'js-select-type')
    select_object02 = Select(select_element02)
    select_object02.select_by_value(type_)

    time.sleep(2)

def select_location(driver, location):
    # Bairro a ser procurado
    # bairro = 'moema'
    # location = 'hortolandia, são paulo'
    # full_address = bairro + ', São Paulo - SP'

    select_element03 = driver.find_element(By.CLASS_NAME, 'js-input')
    select_element03.send_keys(location)

    time.sleep(2)

    driver.implicitly_wait(5)

    search_bairro = driver.find_element(By.CLASS_NAME, 'js-location-name')
    search_bairro.click() 

def get_data(driver):

    # Dataframe to be Filled
    df_final = pd.DataFrame()

    # Get Page Content
    resp = driver.page_source
    soup = BeautifulSoup(resp, 'html.parser')

    # Get URL of each advertisement
    lista_anuncio = []
    for link in soup.find_all('a'):
        url_anuncio = link.get('href')
        if url_anuncio != None and '/imovel/' in url_anuncio:
            lista_anuncio.append(url_anuncio)
    lista_anuncio = list(set(lista_anuncio))
    # print(lista_anuncio)

    url = 'https://www.vivareal.com.br'

    # Acessando cada anuncio e extraindo informações
    for i in lista_anuncio:
         # listas vazias
        list_endereco = []
        list_area = []
        list_quartos = []
        list_banheiro = []
        list_vaga_garagem = []
        list_valor_aluguel = []
        list_valor_cond = []
        list_caracteristicas = []
        list_id_anuncio = []
        list_fonte = []
        
        dict_aux = {}
        
        link_anuncio = url+i
        
        # Getting HTML
        header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'}
        req = Request(link_anuncio, headers = header)
        page = urlopen(req)
        soup = BeautifulSoup(page)
        
        # Getting data
        # try:
        endereco = soup.find('p', class_='title__address js-address')
        endereco = endereco.get_text().strip() if endereco else "sem endereco"
        
        area = soup.find('li', class_='features__item features__item--area js-area')
        area = area.get_text().strip() if area else "sem metragem"
        
        quartos = soup.find('li', class_='features__item features__item--bedroom js-bedrooms')
        quartos = quartos.get_text().strip() if quartos else "sem informacao"
        
        banheiro = soup.find('li', class_='features__item features__item--bathroom js-bathrooms')
        banheiro = banheiro.get_text().strip() if banheiro else "sem informacao"
        
        vaga_garagem = soup.find('li', class_='features__item features__item--parking js-parking')
        vaga_garagem = vaga_garagem.get_text().strip() if vaga_garagem else "sem informcao"
        
        valor_aluguel = soup.find('h3', class_='price__price-info js-price-rent')
        valor_aluguel = valor_aluguel if valor_aluguel != None else soup.find('h3', class_='price__price-info js-price-sale')         
        valor_aluguel = valor_aluguel.get_text().strip() if valor_aluguel else "sem informcao"
        
        valor_cond = soup.find('span', class_='price__list-value condominium js-condominium')
        valor_cond = valor_cond.get_text().strip() if valor_cond else "sem informacao"
        
        caracteristicas = soup.find('ul', class_='amenities__list')
        caracteristicas = caracteristicas.get_text().strip().replace('   ', ', ') if caracteristicas else "sem informacao"
        
        id_anuncio = re.search(r'\d+.$', link_anuncio).group(0).replace('/', '')
        
        fonte = link_anuncio
        # except:
        #     pass
        
        # append listas para criar dataframe
        list_endereco.append(endereco)
        list_area.append(area)
        list_quartos.append(quartos)
        list_banheiro.append(banheiro)
        list_vaga_garagem.append(vaga_garagem)
        list_valor_aluguel.append(valor_aluguel)
        list_valor_cond.append(valor_cond)
        list_caracteristicas.append(caracteristicas)
        list_id_anuncio.append(id_anuncio)
        list_fonte.append(fonte)
        
        dict_aux = {
            'endereco': list_endereco,
            'area': list_area,
            'quartos': list_quartos,
            'banheiros': list_banheiro,
            'vaga_garagem': list_vaga_garagem,
            'valor_aluguel': list_valor_aluguel,
            'valor_cond': list_valor_cond,
            'caracteristicas': list_caracteristicas,
            'id_anuncio': list_id_anuncio,
            'fonte': list_fonte
        }
        
        
        df_final = pd.concat([df_final, pd.DataFrame(dict_aux)]).reset_index(drop=True)
        time.sleep(2)

    return df_final

def click_next_page(driver):
    scroll_down = driver.find_element(By.XPATH, '//*[@id="js-site-main"]/div[2]/section[1]/div[1]/h2')
    webdriver.ActionChains(driver).move_to_element(scroll_down).perform()

    next_page = driver.find_element(By.XPATH, '//*[@id="js-site-main"]/div[2]/div[1]/section/div[2]/div[2]/div/ul/li[9]/a')
    time.sleep(1)
    next_page.click()  
    
    return None
    
    

### WebScrap

In [5]:
# Getting the Driver
driver = main_site()

# ==================================
# ===== FILTERS TO BE APPLIED ======
# ==================================
# Purpose -> 'rent' or 'sale'
select_purpose(driver, 'sale')

# Type = "Apartamento" or "Casa"
select_type(driver, 'Apartamento')

# Location = Bairro/Cidade
select_location(driver, 'São Paulo - SP')

df_final = pd.DataFrame()
while True:
    try:
        # Getting Data From Advertisement of a single page
        df_aux = get_data(driver)

        # concat with final dataframe
        df_final = pd.concat([df_final, df_aux]).reset_index(drop=True)
                
        click_next_page(driver)

        time.sleep(1.5)
        
    except:
        break

In [22]:
while True:
    try:
        # Getting Data From Advertisement of a single page
        df_aux = get_data(driver)

        # concat with final dataframe
        df_final = pd.concat([df_final, df_aux]).reset_index(drop=True)
                
        click_next_page(driver)

        time.sleep(1.5)
        
    except:
        break

In [23]:
df_final.to_csv('df_webscraped.csv')

# BeautifulSoup

## Scripts 01

### Functions

In [247]:
def url_pesquisa(motivo, uf, cidade, tipo, pag):
    url = f'https://www.vivareal.com.br/{motivo}/{uf}/{cidade}/{tipo}/?pagina={str(pag)}'
    return url

def lista_anuncio(url):
    header = {'User-Agent': 'Mozilla/5.0'}
    req = requests.get(url, headers=header)
    soup = BeautifulSoup(req.text, 'html.parser')
    
    lista_anuncio = []
    for link in soup.find_all('a'):
        url_anuncio = link.get('href')
        if url_anuncio != None and '/imovel/' in url_anuncio:
            lista_anuncio.append(url_anuncio)

    lista_anuncio = list(set(lista_anuncio))
    return lista_anuncio

def data_extraction(lista_anuncio):
    base_url = 'https://www.vivareal.com.br'
    
    # percorre cada anúncio extraindo as informações
    for i in anuncios:
        full_url = base_url + i
        
        # Pegando HTML do anuncio
        header = {'User-Agent': 'Mozilla/5.0'}
        r = requests.get(full_url, headers=header)
        soup = BeautifulSoup(r.text, 'html.parser')

        # extraindo informações
        endereco = soup.find('p', class_='title__address js-address')
        endereco = endereco.get_text().strip() if endereco else "sem endereco"
        
        area = soup.find('li', class_='features__item features__item--area js-area')
        area = area.get_text().strip() if area else "sem metragem"
        
        quartos = soup.find('li', class_='features__item features__item--bedroom js-bedrooms')
        quartos = quartos.get_text().strip() if quartos else "sem informacao"
        
        banheiro = soup.find('li', class_='features__item features__item--bathroom js-bathrooms')
        banheiro = banheiro.get_text().strip() if banheiro else "sem informacao"
        
        vaga_garagem = soup.find('li', class_='features__item features__item--parking js-parking')
        vaga_garagem = vaga_garagem.get_text().strip() if vaga_garagem else "sem informcao"
        
        valor_aluguel = soup.find('h3', class_='price__price-info js-price-rent')
        valor_aluguel = valor_aluguel if valor_aluguel != None else soup.find('h3', class_='price__price-info js-price-sale')         
        valor_aluguel = valor_aluguel.get_text().strip() if valor_aluguel else "sem informcao"
        
        valor_cond = soup.find('span', class_='price__list-value condominium js-condominium')
        valor_cond = valor_cond.get_text().strip() if valor_cond else "sem informacao"
        
        caracteristicas = soup.find('ul', class_='amenities__list')
        caracteristicas = caracteristicas.get_text().strip().replace('   ', ', ') if caracteristicas else "sem informacao"
        
        id_anuncio = re.search(r'\d+.$', full_url).group(0).replace('/', '')
        
        fonte = full_url
    
        # appendando informações
        info_dict = {
            'endereco': endereco,
            'area': area,
            'quartos': quartos,
            'banheiro': banheiro,
            'vaga_garagem': vaga_garagem,
            'valor_aluguel': valor_aluguel,
            'valor_cond': valor_cond,
            'caracteristicas':caracteristicas, 
            'id_anuncio': id_anuncio,
            'fonte': fonte,        
        }    
        list_info.append(info_dict)
    
    return list_info

### Script

In [2]:
import re, time, os
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd

In [7]:
# Inicializamos as listas para guardar as informações

link_imovel=[] # nesta lista iremos guardar a url
address=[]     # nesta lista iremos guardar o endereço
neighbor=[]    # nesta lista iremos guardar o bairro
anunciante=[]  # nesta lista iremos guardar o anunciante 
area=[]        # nesta lista iremos guardar a area
tipo=[]        # nesta lista iremos guardar o tipo de imóvel
room=[]        # nesta lista iremos guardar a quantidade de quartos
bath=[]        # nesta lista iremos guardar a quantidade de banheiros
park=[]        # nesta lista iremos guardar a quantidade de vagas de garagem
price=[]       # nesta lista iremos guardar o preço do imóvel

# Ele irá solicitar quantas páginas você deseja coletar
pages_number=int(input('Quantas paginas? '))
# inicializa o tempo de execução
tic = time.time()

# Configure chromedriver
# para executar, é necessário que você baixe o chromedriver e deixe ele na mesma pasta de execução, ou mude o path
# chromedriver = "./chromedriver"
# os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome()

# Informe o link inicial para inicializar o scraping. Você pode trocar entre diversos filtros.
# Este scraper foi desenvolvido para o filtro de alugueis na cidade de santa maria no rio grande do sul. 
# Pode acontecer das informações mudarem caso novos filtros sejam adicionados.
link = f'https://www.vivareal.com.br/aluguel/sp/hortolandia/apartamento_residencial/?pagina=1'
# link = f'https://www.vivareal.com.br/aluguel/rio-grande-do-sul/santa-maria/?pagina=1#onde=BR-Rio_Grande_do_Sul-NULL-Santa_Maria'
driver.get(link)

# Criando o loop entre as paginas do site
for page in range(1,pages_number+1):
   
    # Definimos um sleep time para não sobrecarregar o site
    time.sleep(15)
    # coletamos todas as informações da página e transformamos em formato legivel
    data = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
    soup_complete_source = BeautifulSoup(data.encode('utf-8'), "lxml")
    
    # identificamos todos os itens de card de imóveis
    soup = soup_complete_source.find(class_='results-list js-results-list')    
    

    # Web-Scraping
    # para cada elemento no conjunto de cards, colete:
    for line in soup.findAll(class_="js-card-selector"):
        # colete o endereço completo e o bairro
        try:
            full_address=line.find(class_="property-card__address").text.strip()
            address.append(full_address.replace('\n', '')) #Get all address
            if full_address[:3]=='Rua' or full_address[:7]=='Avenida' or full_address[:8]=='Travessa' or full_address[:7]=='Alameda':
                neighbor_first=full_address.strip().find('-')
                neighbor_second=full_address.strip().find(',', neighbor_first)
                if neighbor_second!=-1:
                    neighbor_text=full_address.strip()[neighbor_first+2:neighbor_second]
                    neighbor.append(neighbor_text) # Guarde na lista todos os bairros
                else: # Bairro não encontrado
                    neighbor_text='-'
                    neighbor.append(neighbor_text) # Caso o bairro não seja encontrado
            else:
                get_comma=full_address.find(',')
                if get_comma!=-1:
                    neighbor_text=full_address[:get_comma]
                    neighbor.append(neighbor_text) # Guarde na lista todos os bairros com problema de formatação provenientes do proprio website  
                else:
                    get_hif=full_address.find('-')
                    neighbor_text=full_address[:get_hif]
                    neighbor.append(neighbor_text)
                    
            # Coleta o link
            full_link=line.find(class_='property-card__main-info').a.get('href')
            link_imovel.append(full_link)
                    
            # Coleta o anunciante
            full_anunciante=line.find(class_='property-card__account-link js-property-card-account-link').img.get('alt').title()
            anunciante.append(full_anunciante)
                    
            # Coleta a área  
            full_area=line.find(class_="property-card__detail-value js-property-card-value property-card__detail-area js-property-card-detail-area").text.strip()
            area.append(full_area)
            
            # Coleta tipologia
            full_tipo = line.find(class_='property-card__title js-cardLink js-card-title').text.split()[0]
            full_tipo=full_tipo.replace(' ','')
            full_tipo=full_tipo.replace('\n','')
            tipo.append(full_tipo)

            # Coleta numero de quartos
            full_room=line.find(class_="property-card__detail-item property-card__detail-room js-property-detail-rooms").text.strip()
            full_room=full_room.replace(' ','')
            full_room=full_room.replace('\n','')
            full_room=full_room.replace('Quartos','')
            full_room=full_room.replace('Quarto','')
            room.append(full_room) #Get apto's rooms

            # Coleta numero de banheiros
            full_bath=line.find(class_="property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom").text.strip()        
            full_bath=full_bath.replace(' ','')
            full_bath=full_bath.replace('\n','')
            full_bath=full_bath.replace('Banheiros','')
            full_bath=full_bath.replace('Banheiro','')
            bath.append(full_bath) #Get apto's Bathrooms

            # Coleta numero de vagas de garagem
            full_park=line.find(class_="property-card__detail-item property-card__detail-garage js-property-detail-garages").text.strip()        
            full_park=full_park.replace(' ','')
            full_park=full_park.replace('\n','')
            full_park=full_park.replace('Vagas','')
            full_park=full_park.replace('Vaga','')
            park.append(full_park) #Get apto's parking lot

            # Coleta preço
            full_price=re.sub('[^0-9]','',line.find(class_="property-card__price js-property-card-prices js-property-card__price-small").text.strip())
            price.append(full_price) #Get apto's parking lot

        except:
            continue
    
    # condicional para parar de iterar entre pages
    if page < pages_number:
        receita = driver.find_element(By.CLASS_NAME, 'js-change-page')
        receita.click()
            
# fecha o chromedriver
driver.quit()

# cria um dataframe pandas e salva como um arquivo CSV
for i in range(0,len(neighbor)):
    combinacao=[link_imovel[i],address[i],neighbor[i],anunciante[i],area[i],tipo[i],room[i],bath[i],park[i],price[i]]
    df=pd.DataFrame(combinacao)
    with open('VivaRealData.csv', 'a', encoding='utf-16', newline='') as f:
        df.transpose().to_csv(f, encoding='iso-8859-1', header=False)

# Tempo de execução
toc = time.time()
get_time=round(toc-tic,3)
print('Finished in ' + str(get_time) + ' seconds')
print(str(len(price))+' results!')

Quantas paginas?  4


ElementClickInterceptedException: Message: element click intercepted: Element <a href="#pagina=" class="js-change-page" title="Página anterior" data-page="" data-disabled="">...</a> is not clickable at point (718, 692). Other element would receive the click: <div class="cookie-notifier__wrapper">...</div>
  (Session info: chrome=97.0.4692.99)
Stacktrace:
Backtrace:
	Ordinal0 [0x00A16903+2517251]
	Ordinal0 [0x009AF8E1+2095329]
	Ordinal0 [0x008B2848+1058888]
	Ordinal0 [0x008E3509+1258761]
	Ordinal0 [0x008E1868+1251432]
	Ordinal0 [0x008DF65D+1242717]
	Ordinal0 [0x008DE4A8+1238184]
	Ordinal0 [0x008D4037+1196087]
	Ordinal0 [0x008F64D3+1336531]
	Ordinal0 [0x008D3A36+1194550]
	Ordinal0 [0x008F65BA+1336762]
	Ordinal0 [0x00905BBF+1399743]
	Ordinal0 [0x008F639B+1336219]
	Ordinal0 [0x008D27A7+1189799]
	Ordinal0 [0x008D3609+1193481]
	GetHandleVerifier [0x00BA5904+1577972]
	GetHandleVerifier [0x00C50B97+2279047]
	GetHandleVerifier [0x00AA6D09+534521]
	GetHandleVerifier [0x00AA5DB9+530601]
	Ordinal0

In [258]:
pip install lxml

  Using cached lxml-4.7.1-cp38-cp38-win_amd64.whl (3.7 MB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
motivo = "aluguel" #aluguel ou venda
uf = "sp" # Estado
cidade = 'hortolandia' 
# zona = 'zona-sul'
# bairro = 'moema'
tipo = 'apartamento_residencial'
pag = 1

list_info = []

# Página inicial da pesquisa
url = url_pesquisa(motivo, uf, cidade, tipo, pag)

# Instanciando Selenium driver
driver = webdriver.Chrome()
# acessar o site Vivareal
driver.get(url)

# pegando lista de anuncios
lista_anuncios = lista_anuncio(url)

# extraindo informações de cada anuncio
data_anuncios = data_extraction(anuncios)

search_bairro = driver.find_element_by_xpath(f'//*[@id="js-site-main"]/div[2]/div[1]/section/div[2]/div[2]/div/ul/li[9]/a')

# search_bairro = driver.find_element(By.CLASS_NAME, 'js-change-page')
search_bairro.click() 







NameError: name 'url_pesquisa' is not defined

In [249]:
pd.DataFrame(data_anuncios)

,endereco,area,quartos,banheiro,vaga_garagem,valor_aluguel,valor_cond,caracteristicas,id_anuncio,fonte
0,"Vila São Pedro, Hortolândia - SP",60m²,2 quartos,2 banheiros 1 suíte,1 vaga,R$ 2.000/Mês,R$ 265,"Piscina, Churrasqueira, Elevador, Varanda, Ar-...",2547759132,https://www.vivareal.com.br/imovel/apartamento...
1,"Jardim Rosolém, Hortolândia - SP",56m²,2 quartos,1 banheiro,1 vaga,R$ 1.200/Mês,R$ 350,"Elevador, Piscina, Quadra poliesportiva",2546751048,https://www.vivareal.com.br/imovel/apartamento...
2,"Terra Preta, Monte Mor - SP",41m²,2 quartos,1 banheiro,1 vaga,R$ 1.000/Mês,sem informacao,"Lavanderia, Cozinha, Garagem, Condomínio fechado",2536306640,https://www.vivareal.com.br/imovel/apartamento...
3,"Jardim Santa Esmeralda, Hortolândia - SP",62m²,2 quartos,1 banheiro,1 vaga,R$ 630/Mês,R$ 298,"Condomínio fechado, Quintal, Área de serviço, ...",2536176891,https://www.vivareal.com.br/imovel/apartamento...
4,"Avenida Thereza Ana Cecon Breda, 0 - Vila São ...",56m²,2 quartos,1 banheiro 1 suíte,1 vaga,R$ 1.100/Mês,R$ 300,"Piscina, Churrasqueira, Elevador, Condomínio f...",2547764958,https://www.vivareal.com.br/imovel/apartamento...
5,"Rua Nelson Pereira Bueno, 124 - Vila São Franc...",51m²,2 quartos,1 banheiro,1 vaga,R$ 1.120/Mês,R$ 330,"Piscina, Churrasqueira, Elevador, Academia, Es...",2540724097,https://www.vivareal.com.br/imovel/apartamento...
6,"Jardim das Colinas, Hortolândia - SP",53m²,2 quartos,2 banheiros 1 suíte,1 vaga,R$ 1.100/Mês,R$ 350,"Churrasqueira, Condomínio fechado, Elevador, I...",2544639254,https://www.vivareal.com.br/imovel/apartamento...
7,"Jardim Santa Clara do Lago II, Hortolândia - SP",56m²,2 quartos,1 banheiro,1 vaga,R$ 1.500/Mês,R$ 310,"Piscina, Churrasqueira, Elevador, Varanda, Var...",2540681642,https://www.vivareal.com.br/imovel/apartamento...
8,"Rua Porto Alegre, 140 - Parque Residencial Joã...",66m²,2 quartos,2 banheiros 1 suíte,2 vagas,R$ 1.380/Mês,R$ 342,"Piscina, Churrasqueira, Elevador, Academia, Va...",2537384382,https://www.vivareal.com.br/imovel/apartamento...
9,"Avenida Thereza Ana Cecon Breda, 0 - Vila São ...",56m²,2 quartos,1 banheiro 1 suíte,1 vaga,R$ 1.100/Mês,R$ 300,"Piscina, Churrasqueira, Elevador, Condomínio f...",2547277325,https://www.vivareal.com.br/imovel/apartamento...


## Scripts 02
Falta resolver mudar os anúncios conforme percorro as páginas

### Functions

In [230]:
import requests

def get_max_page(url):
    header = {'User-Agent': 'Mozilla/5.0'}
    r = requests.get(url, headers = header)

    # Instantiate soup
    soup = BeautifulSoup(r.text, 'html.parser')
    max_page = math.ceil(int(soup.find('strong', class_='results-summary__count js-total-records').get_text().strip().replace('.', '')) / 36)
    return max_page

def url_pesquisa(motivo, uf, cidade, tipo, pag):
    url = f'https://www.vivareal.com.br/{motivo}/{uf}/{cidade}/{tipo}/?pagina={str(pag)}'
    return url

def lista_anuncio(url):
    header = {'User-Agent': 'Mozilla/5.0'}
    req = requests.get(url, headers=header)
    soup = BeautifulSoup(req.text, 'html.parser')
    
    lista_anuncio = []
    for link in soup.find_all('a'):
        url_anuncio = link.get('href')
        if url_anuncio != None and '/imovel/' in url_anuncio:
            lista_anuncio.append(url_anuncio)

    lista_anuncio = list(set(lista_anuncio))
    return lista_anuncio

def data_extraction(lista_anuncio):
    base_url = 'https://www.vivareal.com.br'
    
    # percorre cada anúncio extraindo as informações
    for i in anuncios:
        full_url = base_url + i
        
        # Pegando HTML do anuncio
        header = {'User-Agent': 'Mozilla/5.0'}
        r = requests.get(full_url, headers=header)
        soup = BeautifulSoup(r.text, 'html.parser')

        # extraindo informações
        endereco = soup.find('p', class_='title__address js-address')
        endereco = endereco.get_text().strip() if endereco else "sem endereco"
        
        area = soup.find('li', class_='features__item features__item--area js-area')
        area = area.get_text().strip() if area else "sem metragem"
        
        quartos = soup.find('li', class_='features__item features__item--bedroom js-bedrooms')
        quartos = quartos.get_text().strip() if quartos else "sem informacao"
        
        banheiro = soup.find('li', class_='features__item features__item--bathroom js-bathrooms')
        banheiro = banheiro.get_text().strip() if banheiro else "sem informacao"
        
        vaga_garagem = soup.find('li', class_='features__item features__item--parking js-parking')
        vaga_garagem = vaga_garagem.get_text().strip() if vaga_garagem else "sem informcao"
        
        valor_aluguel = soup.find('h3', class_='price__price-info js-price-rent')
        valor_aluguel = valor_aluguel if valor_aluguel != None else soup.find('h3', class_='price__price-info js-price-sale')         
        valor_aluguel = valor_aluguel.get_text().strip() if valor_aluguel else "sem informcao"
        
        valor_cond = soup.find('span', class_='price__list-value condominium js-condominium')
        valor_cond = valor_cond.get_text().strip() if valor_cond else "sem informacao"
        
        caracteristicas = soup.find('ul', class_='amenities__list')
        caracteristicas = caracteristicas.get_text().strip().replace('   ', ', ') if caracteristicas else "sem informacao"
        
        id_anuncio = re.search(r'\d+.$', full_url).group(0).replace('/', '')
        
        fonte = full_url
    
        # appendando informações
        info_dict = {
            'endereco': endereco,
            'area': area,
            'quartos': quartos,
            'banheiro': banheiro,
            'vaga_garagem': vaga_garagem,
            'valor_aluguel': valor_aluguel,
            'valor_cond': valor_cond,
            'caracteristicas':caracteristicas, 
            'id_anuncio': id_anuncio,
            'fonte': fonte,        
        }    
        list_info.append(info_dict)
    
    return list_info
        


### Scripts

In [239]:
motivo = "aluguel" #aluguel ou venda
uf = "sp" # Estado
cidade = 'hortolandia' 
# zona = 'zona-sul'
# bairro = 'moema'
tipo = 'apartamento_residencial'
pag = 1



# pegando o num máximo de pagina
max_page = get_max_page(url_pesquisa(motivo, uf, cidade, tipo, pag))

list_info = []

for i in range(max_page):

    # pegando html da vitrine
    url = url_pesquisa(motivo, uf, cidade, tipo, i+1)
    print(url)

    # pegando lista de anuncio da vitrine
    anuncios = lista_anuncio(url)

    # extraindo informações dos anuncios
    data_anuncios = data_extraction(anuncios)
    print()

https://www.vivareal.com.br/aluguel/sp/hortolandia/apartamento_residencial/?pagina=1
https://www.vivareal.com.br/aluguel/sp/hortolandia/apartamento_residencial/?pagina=2
https://www.vivareal.com.br/aluguel/sp/hortolandia/apartamento_residencial/?pagina=3
https://www.vivareal.com.br/aluguel/sp/hortolandia/apartamento_residencial/?pagina=4
https://www.vivareal.com.br/aluguel/sp/hortolandia/apartamento_residencial/?pagina=5
https://www.vivareal.com.br/aluguel/sp/hortolandia/apartamento_residencial/?pagina=6
https://www.vivareal.com.br/aluguel/sp/hortolandia/apartamento_residencial/?pagina=7
https://www.vivareal.com.br/aluguel/sp/hortolandia/apartamento_residencial/?pagina=8
https://www.vivareal.com.br/aluguel/sp/hortolandia/apartamento_residencial/?pagina=9


In [228]:
data_anuncios

[{'endereco': 'Vila São Pedro, Hortolândia - SP',
  'area': '60m²',
  'quartos': '2 quartos',
  'banheiro': '2 banheiros     1 suíte',
  'vaga_garagem': '1 vaga',
  'valor_aluguel': 'R$ 2.000/Mês',
  'valor_cond': 'R$ 265',
  'caracteristicas': 'Piscina, Churrasqueira, Elevador, Varanda, Ar-condicionado, Área de serviço, Cozinha, Interfone',
  'id_anuncio': '2547759132',
  'fonte': 'https://www.vivareal.com.br/imovel/apartamento-2-quartos-vila-sao-pedro-bairros-hortolandia-com-garagem-60m2-venda-RS287000-id-2547759132/'},
 {'endereco': 'Jardim Rosolém, Hortolândia - SP',
  'area': '56m²',
  'quartos': '2 quartos',
  'banheiro': '1 banheiro',
  'vaga_garagem': '1 vaga',
  'valor_aluguel': 'R$ 1.200/Mês',
  'valor_cond': 'R$ 350',
  'caracteristicas': 'Elevador, Piscina, Quadra poliesportiva',
  'id_anuncio': '2546751048',
  'fonte': 'https://www.vivareal.com.br/imovel/apartamento-2-quartos-jardim-rosolem-bairros-hortolandia-com-garagem-56m2-aluguel-RS1200-id-2546751048/'},
 {'endereco':

In [237]:
df = pd.DataFrame(data_anuncios)

In [238]:
df.drop_duplicates().shape

(36, 10)

## Scripts 03
Falta resolver mudar os anúncios conforme percorro as páginas

In [12]:
x_dict = {'A': [10], 'B': [20]}

pd.DataFrame(x_dict)

,A,B
0,10,20


In [43]:
# DataFrame to be filled
df_aluguel = pd.DataFrame(columns=['endereco', 'area', 'quartos', 'banheiros', 'vaga_garagem', 'valor_aluguel', 'valor_cond', 'caracteristicas', 'id_anuncio', 'fonte'])

# =======================================
# =============== FILTROS ===============
# =======================================

# motivo = "aluguel" #aluguel ou venda
# uf = "sp" # Estado
# cidade = 'sao-paulo' 
# zona = 'zona-sul'
# bairro = 'moema'
# tipo = 'apartamento_residencial'

motivo = "aluguel" #aluguel ou venda
uf = "sp" # Estado
cidade = 'hortolandia' 
# zona = 'zona-sul'
# bairro = 'moema'
tipo = 'apartamento_residencial'


# =============================================================
# =============== PEGANDO NUM MAXIMO DE PAGINAS ===============
# =============================================================
# url = f'https://www.vivareal.com.br/{motivo}/{uf}/{cidade}/{zona}/{bairro}/{tipo}/?pagina={str(1)}'
url = f'https://www.vivareal.com.br/{motivo}/{uf}/{cidade}/{tipo}/#pagina={str(1)}'
header = {'User-Agent': 'Mozilla/5.0'}
req = Request(url, headers = header)

page = urlopen(req)

# Instantiate soup
soup = BeautifulSoup(page)
max_page = math.ceil(int(soup.find('strong', class_='results-summary__count js-total-records').get_text().strip().replace('.', '')) / 36)

# ======================================================
# =============== PEGANDO URL DA VITRINE ===============
# =============== PAGINA COM 36 ANUNCIOS ===============
# ======================================================
s = requests.Session()
s.headers = {'User-Agent': 'Mozilla/5.0'}

for pag in range(max_page):
#     # url = f'https://www.vivareal.com.br/{motivo}/{uf}/{cidade}/{zona}/{bairro}/{tipo}/?pagina={str(pag+1)}'
#     url = f'https://www.vivareal.com.br/{motivo}/{uf}/{cidade}/{tipo}/?pagina={str(pag)}'
#     print(url)
#     header = {'User-Agent': 'Mozilla/5.0'}
#     req = Request(url, headers = header)

#     page = urlopen(req)

#     # Instantiate soup
#     soup = BeautifulSoup(page)
    url = f'https://www.vivareal.com.br/{motivo}/{uf}/{cidade}/{tipo}/#pagina={str(pag+1)}'
    # header = {'User-Agent': 'Mozilla/5.0'}
    # req = s.get(url, headers=header)
    req = s.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    print(str(pag))
    print(url)

    # ===========================================================
    # =============== PEGANDO URL DE CADA ANÚNCIO ===============
    # ===========================================================
    lista_anuncio = []
    for link in soup.find_all('a'):
        url_anuncio = link.get('href')
        if url_anuncio != None and '/imovel/' in url_anuncio:
            lista_anuncio.append(url_anuncio)

    lista_anuncio = list(set(lista_anuncio))
    print(lista_anuncio)
    
    # ===========================================================================
    # =============== PEGANDO INFORMAÇÕES E SALVANDO EM DATAFRAME ===============
    # ===============         ENTRANDO EM CADA ANUNCIO            ===============
    # ===========================================================================
    url = 'https://www.vivareal.com.br'

    # Acessando cada anuncio e extraindo informações
    for i in lista_anuncio:
         # listas vazias
        list_endereco = []
        list_area = []
        list_quartos = []
        list_banheiro = []
        list_vaga_garagem = []
        list_valor_aluguel = []
        list_valor_cond = []
        list_caracteristicas = []
        list_id_anuncio = []
        list_fonte = []
        
        dict_aux = {}
        
        link_anuncio = url+i
        
        # Getting HTML
        req = Request(link_anuncio, headers = header)
        page = urlopen(req)
        soup = BeautifulSoup(page)
        
        # Getting data
        # try:
        endereco = soup.find('p', class_='title__address js-address')
        endereco = endereco.get_text().strip() if endereco else "sem endereco"
        
        area = soup.find('li', class_='features__item features__item--area js-area')
        area = area.get_text().strip() if area else "sem metragem"
        
        quartos = soup.find('li', class_='features__item features__item--bedroom js-bedrooms')
        quartos = quartos.get_text().strip() if quartos else "sem informacao"
        
        banheiro = soup.find('li', class_='features__item features__item--bathroom js-bathrooms')
        banheiro = banheiro.get_text().strip() if banheiro else "sem informacao"
        
        vaga_garagem = soup.find('li', class_='features__item features__item--parking js-parking')
        vaga_garagem = vaga_garagem.get_text().strip() if vaga_garagem else "sem informcao"
        
        valor_aluguel = soup.find('h3', class_='price__price-info js-price-rent')
        valor_aluguel = valor_aluguel if valor_aluguel != None else soup.find('h3', class_='price__price-info js-price-sale')         
        valor_aluguel = valor_aluguel.get_text().strip() if valor_aluguel else "sem informcao"
        
        valor_cond = soup.find('span', class_='price__list-value condominium js-condominium')
        valor_cond = valor_cond.get_text().strip() if valor_cond else "sem informacao"
        
        caracteristicas = soup.find('ul', class_='amenities__list')
        caracteristicas = caracteristicas.get_text().strip().replace('   ', ', ') if caracteristicas else "sem informacao"
        
        id_anuncio = re.search(r'\d+.$', link_anuncio).group(0).replace('/', '')
        
        fonte = link_anuncio
        # except:
        #     pass
        
        # append listas para criar dataframe
        list_endereco.append(endereco)
        list_area.append(area)
        list_quartos.append(quartos)
        list_banheiro.append(banheiro)
        list_vaga_garagem.append(vaga_garagem)
        list_valor_aluguel.append(valor_aluguel)
        list_valor_cond.append(valor_cond)
        list_caracteristicas.append(caracteristicas)
        list_id_anuncio.append(id_anuncio)
        list_fonte.append(fonte)
        
        dict_aux = {
            'endereco': list_endereco,
            'area': list_area,
            'quartos': list_quartos,
            'banheiros': list_banheiro,
            'vaga_garagem': list_vaga_garagem,
            'valor_aluguel': list_valor_aluguel,
            'valor_cond': list_valor_cond,
            'caracteristicas': list_caracteristicas,
            'id_anuncio': list_id_anuncio,
            'fonte': list_fonte
        }
        
        
        df_aux = pd.DataFrame(dict_aux)
            
        df_aluguel = pd.concat([df_aluguel, df_aux])
        
    print(df_aluguel.shape)
    # df_aluguel = df_aluguel.drop_duplicates()
        

0
https://www.vivareal.com.br/aluguel/sp/hortolandia/apartamento_residencial/#pagina=1
['/imovel/apartamento-3-quartos-vila-sao-francisco-bairros-hortolandia-com-garagem-71m2-aluguel-RS1500-id-2550534456/', '/imovel/apartamento-2-quartos-jardim-nossa-senhora-de-fatima-bairros-hortolandia-com-garagem-43m2-aluguel-RS800-id-2551061203/', '/imovel/apartamento-2-quartos-vila-sao-pedro-bairros-hortolandia-com-garagem-52m2-aluguel-RS1000-id-2546891592/', '/imovel/apartamento-2-quartos-villa-flora-bairros-hortolandia-com-garagem-51m2-venda-RS220000-id-2548508278/', '/imovel/apartamento-2-quartos-jardim-das-colinas-bairros-hortolandia-com-garagem-58m2-aluguel-RS900-id-2550868523/', '/imovel/apartamento-2-quartos-jardim-amanda-i-bairros-hortolandia-com-garagem-40m2-aluguel-RS900-id-2549972521/', '/imovel/apartamento-2-quartos-jardim-nova-hortolandia-i-bairros-hortolandia-com-garagem-49m2-aluguel-RS700-id-2518913696/', '/imovel/apartamento-2-quartos-chacaras-fazenda-coelho-bairros-hortolandia-com

In [42]:
motivo = "aluguel" #aluguel ou venda
uf = "sp" # Estado
cidade = 'hortolandia' 
# zona = 'zona-sul'
# bairro = 'moema'
tipo = 'apartamento_residencial'

s = requests.Session()
s.headers = {'User-Agent': 'Mozilla/5.0'}

pag = 1
max_pag = 3

for pag in range(3):
    url = f'https://www.vivareal.com.br/{motivo}/{uf}/{cidade}/{tipo}/#pagina={str(pag+1)}'
    print(url)
    
    req = s.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')

https://www.vivareal.com.br/aluguel/sp/hortolandia/apartamento_residencial/#pagina=1
https://www.vivareal.com.br/aluguel/sp/hortolandia/apartamento_residencial/#pagina=2
https://www.vivareal.com.br/aluguel/sp/hortolandia/apartamento_residencial/#pagina=3


{'Date': 'Tue, 22 Feb 2022 02:27:01 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'CF-Ray': '6e14c5dfbd324ee9-GRU', 'Age': '902', 'Cache-Control': 'public, max-age=1800', 'Expires': 'Tue, 22 Feb 2022 02:57:01 GMT', 'Last-Modified': 'Tue, 22 Feb 2022 01:39:22 GMT', 'Strict-Transport-Security': 'max-age=15724800', 'Vary': 'Accept-Encoding', 'CF-Cache-Status': 'HIT', 'Expect-CT': 'max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/beacon/expect-ct"', 'X-Content-Type-Options': 'nosniff', 'X-DNS-Prefetch-Control': 'off', 'X-Download-Options': 'noopen', 'X-Frame-Options': 'SAMEORIGIN', 'X-Instance': 'comandante-hamilton-7d65cff465-9bxrf', 'X-XSS-Protection': '1; mode=block', 'Server': 'cloudflare', 'alt-svc': 'h3=":443"; ma=86400, h3-29=":443"; ma=86400'}

In [24]:
df_aluguel.drop_duplicates().shape

(36, 10)

In [23]:
df_aluguel

,endereco,area,quartos,banheiros,vaga_garagem,valor_aluguel,valor_cond,caracteristicas,id_anuncio,fonte
0,"Vila São Francisco, Hortolândia - SP",71m²,3 quartos,2 banheiros 1 suíte,1 vaga,R$ 1.500/Mês,R$ 460,sem informacao,2550534456,https://www.vivareal.com.br/imovel/apartamento...
0,"Jardim Nossa Senhora de Fatima, Hortolândia - SP",43m²,2 quartos,1 banheiro,1 vaga,R$ 800/Mês,R$ 200,sem informacao,2551061203,https://www.vivareal.com.br/imovel/apartamento...
0,"Vila São Pedro, Hortolândia - SP",52m²,2 quartos,1 banheiro,1 vaga,R$ 1.000/Mês,R$ 290,sem informacao,2546891592,https://www.vivareal.com.br/imovel/apartamento...
0,"Jardim São Jorge, Hortolândia - SP",45m²,2 quartos,1 banheiro,1 vaga,R$ 1.050/Mês,sem informacao,"Lavanderia, Cozinha, Garagem, Condomínio fechado",2532168090,https://www.vivareal.com.br/imovel/apartamento...
0,"Rua Joaquim Marcelino Leite, 425 - Villa Flora...",51m²,2 quartos,1 banheiro,1 vaga,R$ 1.150/Mês,R$ 225,"Churrasqueira, Condomínio fechado, Jardim, Pla...",2548508278,https://www.vivareal.com.br/imovel/apartamento...
...,...,...,...,...,...,...,...,...,...,...
0,"Avenida Thereza Ana Cecon Breda, 1881 - Vila S...",58m²,2 quartos,2 banheiros 1 suíte,1 vaga,R$ 1.250/Mês,R$ 300,"Piscina, Churrasqueira, Elevador, Condomínio f...",2548957358,https://www.vivareal.com.br/imovel/apartamento...
0,"Rua José Moscardini, 65 - Jardim Nossa Senhora...",56m²,2 quartos,1 banheiro,1 vaga,R$ 1.200/Mês,R$ 350,"Condomínio fechado, Academia, Espaço gourmet, ...",2551563218,https://www.vivareal.com.br/imovel/apartamento...
0,"Rua Milão, 10 - Jardim Residencial Firenze, Ho...",80m²,3 quartos,2 banheiros 1 suíte,2 vagas,R$ 1.223/Mês,R$ 400,"Condomínio fechado, Portão eletrônico, Aceita ...",2552342659,https://www.vivareal.com.br/imovel/apartamento...
0,"Rua Adamastor Pirschner, 700 - Jardim Santa Cl...",76m²,3 quartos,2 banheiros 1 suíte,2 vagas,R$ 1.300/Mês,R$ 420,"Condomínio fechado, Aceita animais, Playground...",2552279581,https://www.vivareal.com.br/imovel/apartamento...


In [21]:
df_aluguel[df_aluguel['area'] == '71m²']

,endereco,area,quartos,banheiros,vaga_garagem,valor_aluguel,valor_cond,caracteristicas,id_anuncio,fonte
0,"Vila São Francisco, Hortolândia - SP",71m²,3 quartos,2 banheiros 1 suíte,1 vaga,R$ 1.500/Mês,R$ 460,sem informacao,2550534456,https://www.vivareal.com.br/imovel/apartamento...
0,"Vila São Francisco, Hortolândia - SP",71m²,3 quartos,2 banheiros 1 suíte,1 vaga,R$ 1.500/Mês,R$ 460,sem informacao,2550534456,https://www.vivareal.com.br/imovel/apartamento...
0,"Vila São Francisco, Hortolândia - SP",71m²,3 quartos,2 banheiros 1 suíte,1 vaga,R$ 1.500/Mês,R$ 460,sem informacao,2550534456,https://www.vivareal.com.br/imovel/apartamento...
0,"Vila São Francisco, Hortolândia - SP",71m²,3 quartos,2 banheiros 1 suíte,1 vaga,R$ 1.500/Mês,R$ 460,sem informacao,2550534456,https://www.vivareal.com.br/imovel/apartamento...
0,"Vila São Francisco, Hortolândia - SP",71m²,3 quartos,2 banheiros 1 suíte,1 vaga,R$ 1.500/Mês,R$ 460,sem informacao,2550534456,https://www.vivareal.com.br/imovel/apartamento...
0,"Vila São Francisco, Hortolândia - SP",71m²,3 quartos,2 banheiros 1 suíte,1 vaga,R$ 1.500/Mês,R$ 460,sem informacao,2550534456,https://www.vivareal.com.br/imovel/apartamento...
0,"Vila São Francisco, Hortolândia - SP",71m²,3 quartos,2 banheiros 1 suíte,1 vaga,R$ 1.500/Mês,R$ 460,sem informacao,2550534456,https://www.vivareal.com.br/imovel/apartamento...
0,"Vila São Francisco, Hortolândia - SP",71m²,3 quartos,2 banheiros 1 suíte,1 vaga,R$ 1.500/Mês,R$ 460,sem informacao,2550534456,https://www.vivareal.com.br/imovel/apartamento...


In [144]:
df_aluguel[df_aluguel['id_anuncio'] == '2547759132']

,endereco,area,quartos,banheiros,vaga_garagem,valor_aluguel,valor_cond,caracteristicas,id_anuncio,fonte
0,"Vila São Pedro, Hortolândia - SP",60m²,2 quartos,2 banheiros 1 suíte,1 vaga,R$ 2.000/Mês,R$ 265,"Piscina, Churrasqueira, Elevador, Varanda, Ar-...",2547759132,https://www.vivareal.com.br/imovel/apartamento...
0,"Vila São Pedro, Hortolândia - SP",60m²,2 quartos,2 banheiros 1 suíte,1 vaga,R$ 2.000/Mês,R$ 265,"Piscina, Churrasqueira, Elevador, Varanda, Ar-...",2547759132,https://www.vivareal.com.br/imovel/apartamento...
0,"Vila São Pedro, Hortolândia - SP",60m²,2 quartos,2 banheiros 1 suíte,1 vaga,R$ 2.000/Mês,R$ 265,"Piscina, Churrasqueira, Elevador, Varanda, Ar-...",2547759132,https://www.vivareal.com.br/imovel/apartamento...
0,"Vila São Pedro, Hortolândia - SP",60m²,2 quartos,2 banheiros 1 suíte,1 vaga,R$ 2.000/Mês,R$ 265,"Piscina, Churrasqueira, Elevador, Varanda, Ar-...",2547759132,https://www.vivareal.com.br/imovel/apartamento...
0,"Vila São Pedro, Hortolândia - SP",60m²,2 quartos,2 banheiros 1 suíte,1 vaga,R$ 2.000/Mês,R$ 265,"Piscina, Churrasqueira, Elevador, Varanda, Ar-...",2547759132,https://www.vivareal.com.br/imovel/apartamento...
0,"Vila São Pedro, Hortolândia - SP",60m²,2 quartos,2 banheiros 1 suíte,1 vaga,R$ 2.000/Mês,R$ 265,"Piscina, Churrasqueira, Elevador, Varanda, Ar-...",2547759132,https://www.vivareal.com.br/imovel/apartamento...
0,"Vila São Pedro, Hortolândia - SP",60m²,2 quartos,2 banheiros 1 suíte,1 vaga,R$ 2.000/Mês,R$ 265,"Piscina, Churrasqueira, Elevador, Varanda, Ar-...",2547759132,https://www.vivareal.com.br/imovel/apartamento...
0,"Vila São Pedro, Hortolândia - SP",60m²,2 quartos,2 banheiros 1 suíte,1 vaga,R$ 2.000/Mês,R$ 265,"Piscina, Churrasqueira, Elevador, Varanda, Ar-...",2547759132,https://www.vivareal.com.br/imovel/apartamento...
0,"Vila São Pedro, Hortolândia - SP",60m²,2 quartos,2 banheiros 1 suíte,1 vaga,R$ 2.000/Mês,R$ 265,"Piscina, Churrasqueira, Elevador, Varanda, Ar-...",2547759132,https://www.vivareal.com.br/imovel/apartamento...


Proximo passo: verificar o FOR loop, pois está retornando um dataframe apenas com as informações da última página